In [1]:
import os
from pathlib import Path

In [2]:
path_current = os.getcwd()
path_main = str(Path(path_current).parent)

### To main directory

In [3]:
# change directory to main path
os.chdir(path_main)

### Load libraries, functions, palette, theme

In [4]:
# load libraries and palette
%run __libraries.ipynb

In [5]:
# load functions
%run __functions.ipynb

In [6]:
# load regions data
path_regions_data = path_main + '/files'
path_change(path_regions_data)

%run regions_data.ipynb

In [7]:
# start stopwatch
start = stopwatch_start()

### Variables

In [8]:
path_files = path_main + '/files'

In [9]:
path_excel_csv = path_main + '/excel-csv'

### Load Files

In [10]:
economics_data = loadit('economics_data', path=path_files, create_empty_dict=True)

### Back to notebook directory

In [11]:
# back to current path
os.chdir(path_current)

# <font color='#2C8B6D'>II. Цены, инфляция</font>

# Section II. Цены производителей

## 10. Сельскохозяйственная продукция

### 10.1. Средние цены производителей сельскохозяйственной продукции (с 2010 г.) - до 2022

*Источник: https://rosstat.gov.ru/storage/mediabank/cena_sx-09.xlsx*  
*Единицы измерения: *

*Примечания:*  

In [12]:
replace_dict_partial = {
    '1\\)': '',
    '2\\)': '',
    ',': '.'
}
replace_dict = {
    '...': np.NaN,
    '…': np.NaN,
    '..': np.NaN,
    '-': np.NaN,
    ' ': np.NaN
}

#### 2024 year link formula

In [13]:
for i in range(1,13):
    link = f'https://rosstat.gov.ru/storage/mediabank/cena_sx_' + str(i).zfill(2) + '-2024.xlsx'
    link_next = f'https://rosstat.gov.ru/storage/mediabank/cena_sx_' + str(i+1).zfill(2) + '-2024.xlsx'
    response = requests.get(link)
    response_next = requests.get(link_next)
    condition = ((response.status_code == 200) & (response_next.status_code == 404))
    if condition:
        pci_producers_link = link

In [14]:
pci_producers_link

'https://rosstat.gov.ru/storage/mediabank/cena_sx_04-2024.xlsx'

In [15]:
pci_producers_raw = pd.read_excel(pci_producers_link, sheet_name='1')

In [16]:
pci_producers_raw

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Средние цены производителей сельскохозяйственн...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"на конец периода, рублей за тонну (яйца - за т...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2010,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022 1,2023 1
3,Зерновые культуры,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,пшеница,3867,5108.0,6409.0,6715.0,6849.0,8768.0,8837.0,7304.0,8537.0,10279.0,12240.0,14265.0,12813,10847
5,кукуруза,4681,5917.0,6751.0,6581.0,5799.0,7853.0,8348.0,7030.0,7910.0,10059.0,10881.0,13788.0,13215,10261
6,ячмень,3395,4986.0,5903.0,6376.0,5516.0,7344.0,7741.0,6782.0,8115.0,9552.0,9742.0,12844.0,12795,9729
7,рожь,3411,3924.0,4519.0,4912.0,4691.0,5247.0,6149.0,5622.0,5561.0,8073.0,9662.0,10115.0,10634,7423
8,овес,3596,4495.0,4597.0,5782.0,4965.0,5493.0,6400.0,6520.0,5970.0,6877.0,8510.0,10746.0,10574,8075
9,просо,3832,5158.0,3982.0,5241.0,5609.0,7365.0,6439.0,5102.0,11783.0,13387.0,11736.0,12201.0,11620,12118


In [17]:
pci_producers = pci_producers_raw[2:-2].copy()

In [18]:
pci_producers.iloc[0,0] = '-'

In [19]:
# drop duplicate whitespaces
pci_producers.iloc[:, 0] = \
    pci_producers.iloc[:, 0].apply(lambda x: remove_duplicated_whitespaces(x))

In [20]:
pci_producers.columns = pci_producers.iloc[0, :]

In [21]:
pci_producers = pci_producers[2:].copy()

In [22]:
pci_producers.columns = [str(i)[:4] for i in pci_producers.columns]

In [23]:
pci_producers.columns = [to_int(i) for i in pci_producers.columns]

In [24]:
pci_producers = pci_producers.drop([13, 21], axis=0)

In [25]:
pci_producers = pci_producers.set_index('-', drop=True)

In [26]:
pci_producers.index.name = None

In [27]:
pci_producers.index = [str(i).lower() for i in pci_producers.index]

In [28]:
pci_producers = pci_producers.T

In [29]:
pci_producers = pci_producers.astype(float)

In [30]:
pci_producers = pci_producers.round(1)

In [31]:
pci_producers.head()

,пшеница,кукуруза,ячмень,рожь,овес,просо,гречиха,овощи бобовые сушеные (культуры зернобобовые),семена подсолнечника,капуста всех сортов,...,морковь столовая,лук репчатый,свекла столовая,картофель,крупный рогатый скот,овцы и козы,свиньи,птица сельскохозяйственная живая,молоко сырое крупного рогатого скота,яйца куриные в скорлупе свежие
2010,3867.0,4681.0,3395.0,3411.0,3596.0,3832.0,8153.0,5581.0,10605.0,11029.0,...,11309.0,9719.0,8298.0,9501.0,55951.0,45174.0,69748.0,52966.0,12370.0,2341.0
2011,5108.0,5917.0,4986.0,3924.0,4495.0,5158.0,15676.0,6991.0,11364.0,10842.0,...,12232.0,9101.0,11225.0,10308.0,64989.0,50902.0,76386.0,54547.0,14135.0,2537.0
2012,6409.0,6751.0,5903.0,4519.0,4597.0,3982.0,10537.0,8335.0,12458.0,6503.0,...,7503.0,5730.0,6614.0,7642.0,73097.0,62902.0,83243.0,55173.0,13604.0,2704.0
2013,6715.0,6581.0,6376.0,4912.0,5782.0,5241.0,7205.0,8395.0,12024.0,8227.0,...,9812.0,7306.0,8981.0,9447.0,72128.0,74286.0,71675.0,54358.0,15875.0,3197.0
2014,6849.0,5799.0,5516.0,4691.0,4965.0,5609.0,8370.0,8458.0,11534.0,10123.0,...,11481.0,10595.0,10403.0,12898.0,74405.0,70978.0,94081.0,63654.0,19614.0,3407.0


### 10.2. Средние цены производителей сельскохозяйственной продукции - 2021 год

*Источник: https://rosstat.gov.ru/storage/mediabank/cena_sx-09.xlsx*  
*Единицы измерения: *

*Примечания:*  

In [32]:
pci_producers2021_raw = pd.read_excel(pci_producers_link, sheet_name='2')

In [33]:
pci_producers2021_raw

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Средние цены производителей сельскохозяйственн...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"на конец периода, рублей за тонну (яйца - за т...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ОКПД2,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь
3,Зерновые и зернобобовые культуры,01.11.12.001.АГ,14237,14778,14102,13805,13691,13868,13534,12872,14051,14785,14996,15100
4,Культуры зерновые,01.11.12.003.АГ,14184,14722,14003,13601,13458,13703,13322,12651,13873,14562,14852,14981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Мед натуральный пчелиный,01.49.21.110,245665,239113,246036,269239,247794,251154,243079,281832,268289,252372,269479,275004
57,Шкурки невыделанные норки клеточного разведения,01.49.19.003.АГ,1595,2029,1945,2390,1601,1622,…,…,1420,1928,2119,2488
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,... Данные не размещаются в целях обеспечени...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
pci_producers2021 = pci_producers2021_raw[2:-3].copy()

In [35]:
pci_producers2021 = pci_producers2021.drop(pci_producers2021.columns[1], axis=1)

In [36]:
# drop duplicated whiespaces
pci_producers2021.iloc[:, 0] = \
    [str.join(' ', str(i).split()) for i in pci_producers2021.iloc[:, 0]]

In [37]:
pci_producers2021.columns = pci_producers2021.iloc[0,:]

In [38]:
pci_producers2021.columns.name = None

In [39]:
pci_producers2021 = pci_producers2021[1:].copy()

In [40]:
pci_producers2021.index = pci_producers2021.iloc[:, 0]

In [41]:
pci_producers2021.index = [i.lower() for i in pci_producers2021.index]

In [42]:
pci_producers2021.index.name = None

In [43]:
pci_producers2021 = pci_producers2021.iloc[:,1:].copy()

In [44]:
pci_producers2021 = pci_producers2021.T

In [45]:
pci_producers2021 = \
    pci_producers2021.replace({'…': np.NaN, '-': np.NaN, ' ': '.'}, regex=True)

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_59245/925379630.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pci_producers2021.replace({'…': np.NaN, '-': np.NaN, ' ': '.'}, regex=True)


In [46]:
pci_producers2021 = pci_producers2021.astype(float)

In [47]:
pci_producers2021 = pci_producers2021.round(1)

In [48]:
pci_producers2021.index = [i.replace(' 1', '') for i in pci_producers2021.index]

In [49]:
pci_producers2021.index = [i.lower() for i in pci_producers2021.index]

In [50]:
pci_producers2021.index = [i.strip() for i in pci_producers2021.index]

In [51]:
pci_producers2021.index = \
    (months_translate(x, add_year=2021) for x in pci_producers2021.index)

In [52]:
pci_producers2021

,зерновые и зернобобовые культуры,культуры зерновые,пшеница,пшеница твердая,"пшеница, кроме твердой пшеницы",пшеница мягкая 1 класса,пшеница мягкая 2 класса,пшеница мягкая 3 класса,пшеница мягкая 4 класса,пшеница мягкая 5 класса,...,молоко сырое крупного рогатого скота,лошади живые,овцы и козы,"шерсть стриженая немытая овец и коз, включая стриженую шерсть, промытую руном",свиньи,птица сельскохозяйственная живая,яйца куриные в скорлупе свежие,олени северные,мед натуральный пчелиный,шкурки невыделанные норки клеточного разведения
January 2021,14237.0,14184.0,14810.0,16186.0,14778.0,12460.0,NaN,14670.0,15770.0,13609.0,...,27296.0,104074.0,85627.0,38337.0,97723.0,78839.0,5745.0,140740.0,245665.0,1595.0
February 2021,14778.0,14722.0,14974.0,17930.0,14956.0,12630.0,NaN,15361.0,15323.0,13914.0,...,27392.0,120930.0,96380.0,NaN,100615.0,80991.0,5749.0,122515.0,239113.0,2029.0
March 2021,14102.0,14003.0,14095.0,19750.0,14039.0,12720.0,NaN,14468.0,14344.0,13279.0,...,27393.0,103153.0,103216.0,NaN,104502.0,87277.0,6099.0,160436.0,246036.0,1945.0
April 2021,13805.0,13601.0,13317.0,24231.0,13213.0,NaN,NaN,13541.0,13302.0,12871.0,...,27237.0,102950.0,107597.0,NaN,106586.0,92424.0,6753.0,108285.0,269239.0,2390.0
May 2021,13691.0,13458.0,13140.0,19519.0,13074.0,NaN,NaN,13404.0,13166.0,12754.0,...,26945.0,96923.0,97340.0,NaN,111958.0,92731.0,6162.0,85245.0,247794.0,1601.0
June 2021,13868.0,13703.0,13415.0,19369.0,13363.0,NaN,NaN,13688.0,13313.0,13075.0,...,26748.0,83335.0,111049.0,NaN,113162.0,89558.0,4750.0,147496.0,251154.0,1622.0
July 2021,13534.0,13322.0,13179.0,17730.0,13136.0,NaN,NaN,13353.0,13054.0,12995.0,...,26633.0,94776.0,113959.0,145383.0,114977.0,89714.0,4500.0,135205.0,243079.0,NaN
August 2021,12872.0,12651.0,12826.0,16888.0,12798.0,NaN,NaN,13305.0,13021.0,11964.0,...,26720.0,123612.0,112642.0,106116.0,119961.0,90926.0,4493.0,96943.0,281832.0,NaN
September 2021,14051.0,13873.0,14173.0,15827.0,14137.0,15896.0,NaN,14598.0,14450.0,13080.0,...,27067.0,99536.0,122854.0,116670.0,123787.0,93498.0,4666.0,122882.0,268289.0,1420.0
October 2021,14785.0,14562.0,14705.0,18834.0,14616.0,NaN,NaN,15162.0,15006.0,13575.0,...,27917.0,129690.0,93021.0,110233.0,124618.0,98049.0,5579.0,157464.0,252372.0,1928.0


### 10.3. Средние цены производителей сельскохозяйственной продукции - 2022 год

*Источник: https://rosstat.gov.ru/storage/mediabank/cena_sx-09.xlsx*  
*Единицы измерения: *

*Примечания:*  

In [53]:
pci_producers2022_raw = pd.read_excel(pci_producers_link, sheet_name='3')

In [54]:
pci_producers2022_raw

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Средние цены производителей сел...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"на конец периода, рублей за тонну (яйца - за т...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ОКПД2,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь 1,Ноябрь 1,Декабрь 1
3,Зерновые и зернобобовые культуры,01.11.12.001.АГ,15467,15717,15669,16259,16322,15570,14575,12951,11689,10959,11009,10976
4,Культуры зерновые,01.11.12.003.АГ,15267,15401,15484,16089,16180,15438,14444,12862,11479,10834,10900,10830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Шкурки невыделанные норки \nклеточного разве...,01.49.19.003.АГ,3286,2339,2661,1546,2074,…,1416,-,1556,1472,1724,2090
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,1 Без учета статистической информации по Донец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,... Данные не размещаются в целях обеспечени...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
pci_producers2022 = pci_producers2022_raw[2:-4].copy()

In [56]:
pci_producers2022 = pci_producers2022.drop(pci_producers2022.columns[1], axis=1)

In [57]:
# drop duplicated whiespaces
pci_producers2022.iloc[:, 0] = \
    [str.join(' ', str(i).split()) for i in pci_producers2022.iloc[:, 0]]

In [58]:
pci_producers2022.columns = pci_producers2022.iloc[0,:]

In [59]:
pci_producers2022.columns.name = None

In [60]:
pci_producers2022 = pci_producers2022[1:].copy()

In [61]:
pci_producers2022.index = pci_producers2022.iloc[:, 0]

In [62]:
pci_producers2022.index = [i.lower() for i in pci_producers2022.index]

In [63]:
pci_producers2022.index.name = None

In [64]:
pci_producers2022 = pci_producers2022.iloc[:, 1:].copy()

In [65]:
pci_producers2022 = pci_producers2022.T

In [66]:
pci_producers2022 = \
    pci_producers2022.replace({'…': np.NaN, '-': np.NaN, ' ': '.'}, regex=True)

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_59245/1122602105.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pci_producers2022.replace({'…': np.NaN, '-': np.NaN, ' ': '.'}, regex=True)


In [67]:
pci_producers2022 = pci_producers2022.astype(float)

In [68]:
pci_producers2022 = pci_producers2022.round(1)

In [69]:
pci_producers2022.index = [i.replace(' 1', '') for i in pci_producers2022.index]

In [70]:
pci_producers2022.index = [i.lower() for i in pci_producers2022.index]

In [71]:
pci_producers2022.index = [i.strip() for i in pci_producers2022.index]

In [72]:
pci_producers2022.index = \
    (months_translate(x, add_year=2022) for x in pci_producers2022.index)

In [73]:
pci_producers2022

,зерновые и зернобобовые культуры,культуры зерновые,пшеница,пшеница твердая,"пшеница, кроме твердой пшеницы",пшеница мягкая 1 класса,пшеница мягкая 2 класса,пшеница мягкая 3 класса,пшеница мягкая 4 класса,пшеница мягкая 5 класса,...,молоко сырое крупного рогатого скота,лошади живые,овцы и козы,"шерсть стриженая немытая овец и коз, включая стриженую шерсть, промытую руном",свиньи,птица сельскохозяйственная живая,яйца куриные в скорлупе свежие,олени северные,мед натуральный пчелиный,шкурки невыделанные норки клеточного разведения
January 2022,15467.0,15267.0,15360.0,27561.0,15193.0,13296.0,14538.0,15212.0,15538.0,14444.0,...,30642.0,103195.0,107911.0,NaN,107206.0,102211.0,6142.0,137901.0,184691.0,3286.0
February 2022,15717.0,15401.0,15501.0,31410.0,15240.0,NaN,NaN,15140.0,15441.0,15114.0,...,31681.0,159800.0,103699.0,NaN,103061.0,100261.0,5987.0,113866.0,273541.0,2339.0
March 2022,15669.0,15484.0,15267.0,29790.0,15157.0,NaN,NaN,15286.0,15108.0,14949.0,...,32272.0,134311.0,120030.0,NaN,107297.0,99526.0,6336.0,128024.0,335615.0,2661.0
April 2022,16259.0,16089.0,15982.0,23688.0,15919.0,NaN,14438.0,16115.0,16157.0,15319.0,...,32776.0,216496.0,123367.0,NaN,113936.0,105233.0,6292.0,115839.0,296080.0,1546.0
May 2022,16322.0,16180.0,16164.0,21238.0,16133.0,NaN,17649.0,16407.0,16122.0,15666.0,...,33031.0,138046.0,122417.0,NaN,109093.0,106281.0,5143.0,90317.0,253868.0,2074.0
June 2022,15570.0,15438.0,15495.0,22074.0,15408.0,NaN,NaN,15786.0,15063.0,15312.0,...,33067.0,139498.0,115664.0,NaN,103645.0,103627.0,4755.0,117272.0,301126.0,NaN
July 2022,14575.0,14444.0,14504.0,26062.0,14441.0,NaN,NaN,14613.0,14253.0,14337.0,...,32971.0,121638.0,121488.0,92487.0,104761.0,101941.0,4616.0,137001.0,337795.0,1416.0
August 2022,12951.0,12862.0,12575.0,12233.0,12577.0,NaN,NaN,13480.0,12557.0,11974.0,...,33173.0,266239.0,118059.0,148025.0,110259.0,102272.0,4690.0,146966.0,298705.0,NaN
September 2022,11689.0,11479.0,11209.0,15829.0,11183.0,NaN,NaN,12302.0,11568.0,10373.0,...,33425.0,112241.0,109894.0,NaN,110743.0,102536.0,5011.0,160871.0,313353.0,1556.0
October 2022,10959.0,10834.0,10584.0,15389.0,10542.0,NaN,NaN,11649.0,11258.0,9300.0,...,33916.0,146860.0,95322.0,NaN,108579.0,98942.0,5517.0,189857.0,340698.0,1472.0


### 10.4. Средние цены производителей сельскохозяйственной продукции - 2023 год

*Источник: https://rosstat.gov.ru/storage/mediabank/cena_sx-09.xlsx*  
*Единицы измерения: *

*Примечания:*  

In [74]:
pci_producers2023_raw = pd.read_excel(pci_producers_link, sheet_name='4')

In [75]:
pci_producers2023_raw

,Средние цены производителей сельскохозяйственной продукции \nпо Российской Федерации в 2023 г.1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,"на конец периода, рублей за тонну (яйца - за т...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,ОКПД2,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь
2,Зерновые и зернобобовые культуры,01.11.12.001.АГ,11281,11321,11416,11148,10691,10434,10389,11385,11396,11457,10621,10406
3,Культуры зерновые,01.11.12.003.АГ,11158,11232,11274,10898,10466,10230,10249,11161,11060,10973,10392,10055
4,Пшеница,01.11.1,11266,11530,11351,10928,10350,10096,10178,11411,11063,10885,10399,10295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Шкурки невыделанные норки \n клеточного раз...,01.49.19.003.АГ,2514,1937,2152,1829,1211,1695,…,…,1860,1966,…,1828
57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,1 Без учета статистической информации по Донец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,... Данные не размещаются в целях обеспечени...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
pci_producers2023 = pci_producers2023_raw[1:-3].copy()

In [77]:
pci_producers2023 = pci_producers2023.drop(pci_producers2023.columns[1], axis=1)

In [78]:
# drop duplicated whiespaces
pci_producers2023.iloc[:, 0] = \
    [str.join(' ', str(i).split()) for i in pci_producers2023.iloc[:, 0]]

In [79]:
pci_producers2023.columns = pci_producers2023.iloc[0,:]

In [80]:
pci_producers2023.columns.name = None

In [81]:
pci_producers2023 = pci_producers2023[1:].copy()

In [82]:
pci_producers2023.index = pci_producers2023.iloc[:, 0]

In [83]:
pci_producers2023.index = [i.lower() for i in pci_producers2023.index]

In [84]:
pci_producers2023.index.name = None

In [85]:
pci_producers2023 = pci_producers2023.iloc[:,1:].copy()

In [86]:
pci_producers2023 = pci_producers2023.T

In [87]:
pci_producers2023 = pci_producers2023.replace(
    {'…': np.NaN, '-': np.NaN, ' ': '.'}, regex=True)

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_59245/2432282431.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pci_producers2023 = pci_producers2023.replace(


In [88]:
pci_producers2023 = pci_producers2023.astype(float)

In [89]:
pci_producers2023 = pci_producers2023.round(1)

In [90]:
pci_producers2023.index = [i.lower() for i in pci_producers2023.index]

In [91]:
pci_producers2023.index = [i.strip() for i in pci_producers2023.index]

In [92]:
pci_producers2023.index = \
    (months_translate(x, add_year=2023) for x in pci_producers2023.index)

In [93]:
pci_producers2023

,зерновые и зернобобовые культуры,культуры зерновые,пшеница,пшеница твердая,"пшеница, кроме твердой пшеницы",пшеница мягкая 1 класса,пшеница мягкая 2 класса,пшеница мягкая 3 класса,пшеница мягкая 4 класса,пшеница мягкая 5 класса,...,лошади живые,овцы и козы,"шерсть стриженая немытая овец и коз, включая стриженую шерсть, промытую руном",свиньи,птица сельскохозяйственная живая,яйца куриные в скорлупе свежие,олени северные,мед натуральный пчелиный,шкурки невыделанные норки клеточного разведения,nan
January 2023,11281.0,11158.0,11266.0,14614.0,11231.0,NaN,NaN,12624.0,11613.0,9612.0,...,126808.0,150250.0,10500.0,104422.0,97968.0,5823.0,113628.0,234313.0,2514.0,NaN
February 2023,11321.0,11232.0,11530.0,13705.0,11503.0,NaN,9107.0,12328.0,11888.0,10011.0,...,128344.0,111121.0,NaN,100399.0,95455.0,5901.0,140678.0,314506.0,1937.0,NaN
March 2023,11416.0,11274.0,11351.0,13574.0,11333.0,NaN,NaN,12425.0,11481.0,9702.0,...,177541.0,119157.0,NaN,102536.0,95493.0,6257.0,170093.0,326886.0,2152.0,NaN
April 2023,11148.0,10898.0,10928.0,11856.0,10918.0,NaN,NaN,11594.0,11399.0,9566.0,...,145159.0,124671.0,NaN,105434.0,96386.0,6253.0,146671.0,342625.0,1829.0,NaN
May 2023,10691.0,10466.0,10350.0,11782.0,10337.0,NaN,NaN,10872.0,10769.0,9217.0,...,134011.0,151228.0,NaN,106936.0,101559.0,5512.0,105070.0,258176.0,1211.0,NaN
June 2023,10434.0,10230.0,10096.0,11940.0,10080.0,NaN,NaN,10588.0,10344.0,9339.0,...,140278.0,136255.0,NaN,108431.0,107604.0,5236.0,75438.0,319555.0,1695.0,NaN
July 2023,10389.0,10249.0,10178.0,13482.0,10127.0,NaN,NaN,10831.0,10649.0,8831.0,...,113533.0,136153.0,55389.0,114067.0,113720.0,5479.0,75054.0,388421.0,NaN,NaN
August 2023,11385.0,11161.0,11411.0,15217.0,11300.0,NaN,10005.0,12277.0,11966.0,9233.0,...,144308.0,131762.0,NaN,124288.0,118750.0,5636.0,46379.0,303356.0,NaN,NaN
September 2023,11396.0,11060.0,11063.0,17786.0,10948.0,NaN,NaN,12138.0,12119.0,9354.0,...,154093.0,129501.0,NaN,126587.0,123383.0,6078.0,101202.0,347863.0,1860.0,NaN
October 2023,11457.0,10973.0,10885.0,17366.0,10786.0,NaN,NaN,12114.0,11616.0,9201.0,...,175341.0,130077.0,NaN,127935.0,130675.0,7182.0,82075.0,216812.0,1966.0,NaN


### 13.01. Средние цены производителей сельскохозяйственной продукции - 2024 год

*Источник: https://rosstat.gov.ru/storage/mediabank/cena_sx-09.xlsx*  
*Единицы измерения: *

*Примечания:*  

In [94]:
pci_producers2024_raw = pd.read_excel(pci_producers_link, sheet_name='5')

In [95]:
pci_producers2024_raw

,Средние цены производителей сельскохозяйственной продукции \nпо Российской Федерации в 2024 г.1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,"на конец периода, рублей за тонну (яйца - за т...",NaN,NaN,NaN,NaN,NaN
1,NaN,ОКПД2,Январь,Февраль,Март,Апрель
2,Зерновые и зернобобовые культуры,01.11.12.001.АГ,10891.85,10944.01,10888.22,10712.65
3,Культуры зерновые,01.11.12.003.АГ,10392.99,10605.28,10601.85,10487.59
4,Пшеница,01.11.1,10376.72,10470.67,10338.75,10424.37
...,...,...,...,...,...,...
61,Шкурки невыделанные норки клеточного разведения,01.49.19.003.АГ,2638.89,2682.77,2572.54,2092.6
62,Шкурки невыделанные соболя клеточного развед...,01.49.19.005.АГ,...,-,-,…
63,1 Без учета статистической информации по Доне...,NaN,NaN,NaN,NaN,NaN
64,... Данные не размещаются в целях обеспечени...,NaN,NaN,NaN,NaN,NaN


In [96]:
pci_producers2024 = pci_producers2024_raw[1:-4].copy()

In [97]:
pci_producers2024 = pci_producers2024.drop(pci_producers2024.columns[1], axis=1)

In [98]:
pci_producers2024.iloc[:, 0] = \
    pci_producers2024.iloc[:, 0].apply(lambda x: remove_duplicated_whitespaces(x))

In [99]:
pci_producers2024.columns = pci_producers2024.iloc[0,:]

In [100]:
pci_producers2024.columns.name = None

In [101]:
pci_producers2024 = pci_producers2024[1:].copy()

In [102]:
pci_producers2024.index = pci_producers2024.iloc[:, 0]

In [103]:
pci_producers2024.index = [i.lower() for i in pci_producers2024.index]

In [104]:
pci_producers2024.index.name = None

In [105]:
pci_producers2024 = pci_producers2024.iloc[:, 1:].copy()

In [106]:
pci_producers2024 = pci_producers2024.T

In [107]:
pci_producers2024 = pci_producers2024.replace(replace_dict_partial, regex=True)

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_59245/2102754835.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pci_producers2024 = pci_producers2024.replace(replace_dict_partial, regex=True)


In [108]:
pci_producers2024 = pci_producers2024.replace(replace_dict)

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_59245/1258666742.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pci_producers2024 = pci_producers2024.replace(replace_dict)


In [109]:
pci_producers2024 = pci_producers2024.astype(float)

In [110]:
pci_producers2024 = pci_producers2024.round(1)

In [111]:
pci_producers2024.index = [i.lower() for i in pci_producers2024.index]

In [112]:
pci_producers2024.index = [i.strip() for i in pci_producers2024.index]

In [113]:
pci_producers2024.index = [i.replace('2', '') for i in pci_producers2024.index]

In [114]:
pci_producers2024.index = [i.replace(')', '') for i in pci_producers2024.index]

In [115]:
pci_producers2024.index = \
    (months_translate(x, add_year=2024) for x in pci_producers2024.index)

In [116]:
pci_producers2024

,зерновые и зернобобовые культуры,культуры зерновые,пшеница,пшеница твердая,"пшеница, кроме твердой пшеницы",пшеница мягкая 1 класса,пшеница мягкая 2 класса,пшеница мягкая 3 класса,пшеница мягкая 4 класса,пшеница мягкая 5 класса,...,овцы и козы,"шерсть стриженая немытая овец и коз, включая стриженую шерсть, промытую руном",свиньи,птица сельскохозяйственная живая,яйца куриные в скорлупе свежие,олени северные,мед натуральный пчелиный,шкурки невыделанные лисицы клеточного разведения,шкурки невыделанные песца клеточного разведения,шкурки невыделанные норки клеточного разведения
January 2024,10891.8,10393.0,10376.7,21827.5,10292.2,NaN,NaN,12084.6,10497.4,8927.3,...,98812.3,10191.1,113121.5,122245.6,9747.3,164856.4,359005.7,NaN,NaN,2638.9
February 2024,10944.0,10605.3,10470.7,18331.4,10426.8,NaN,NaN,12247.0,10691.4,8827.7,...,145654.5,NaN,115862.0,117156.8,9575.4,137772.0,230473.9,NaN,NaN,2682.8
March 2024,10888.2,10601.8,10338.8,21029.6,10280.2,NaN,NaN,12357.9,10619.0,8681.0,...,163180.0,NaN,119747.3,113325.8,9235.3,195291.7,469935.1,NaN,NaN,2572.5
April 2024,10712.6,10487.6,10424.4,17643.8,10400.7,10101.5,NaN,12058.5,10614.9,8732.4,...,176090.8,NaN,114468.2,114793.5,9185.8,174577.7,452363.3,NaN,NaN,2092.6


### Combibnig datasets

In [132]:
cp_producers = (pci_producers2021.T
          .join(pci_producers2022.T, how='outer')
          .join(pci_producers2023.T, how='outer')
          .join(pci_producers2024.T, how='outer')).T.copy()

In [133]:
len(cp_producers.T)

61

In [134]:
cp_producers.head()

,nan,бобы соевые,виноград,волокно льна-долгунца,горох сушеный,гречиха,зерновые и зернобобовые культуры,капуста всех сортов,картофель,корнеплоды свеклы сахарной,...,томаты (помидоры) открытого грунта,тритикале,чеснок,"шерсть стриженая немытая овец и коз, включая стриженую шерсть, промытую руном",шкурки невыделанные лисицы клеточного разведения,шкурки невыделанные норки клеточного разведения,шкурки невыделанные песца клеточного разведения,ягоды и плоды растений вида vaccinium,яйца куриные в скорлупе свежие,ячмень
January 2021,NaN,31044.0,NaN,32560.0,17063.0,32160.0,14237.0,11806.0,14620.0,3409.0,...,NaN,11161.0,NaN,38337.0,NaN,1595.0,NaN,NaN,5745.0,10727.0
February 2021,NaN,35186.0,NaN,38089.0,16840.0,34399.0,14778.0,12704.0,15820.0,2963.0,...,NaN,11189.0,NaN,NaN,NaN,2029.0,NaN,NaN,5749.0,12224.0
March 2021,NaN,40411.0,NaN,33161.0,17464.0,35788.0,14102.0,13892.0,17398.0,2945.0,...,NaN,13049.0,NaN,NaN,NaN,1945.0,NaN,NaN,6099.0,12633.0
April 2021,NaN,42407.0,NaN,37279.0,19506.0,37233.0,13805.0,12803.0,17018.0,NaN,...,NaN,12122.0,NaN,NaN,NaN,2390.0,NaN,NaN,6753.0,12603.0
May 2021,NaN,43362.0,NaN,33482.0,21085.0,38840.0,13691.0,12605.0,17976.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1601.0,NaN,NaN,6162.0,13337.0


In [135]:
cp_producers.tail()

,nan,бобы соевые,виноград,волокно льна-долгунца,горох сушеный,гречиха,зерновые и зернобобовые культуры,капуста всех сортов,картофель,корнеплоды свеклы сахарной,...,томаты (помидоры) открытого грунта,тритикале,чеснок,"шерсть стриженая немытая овец и коз, включая стриженую шерсть, промытую руном",шкурки невыделанные лисицы клеточного разведения,шкурки невыделанные норки клеточного разведения,шкурки невыделанные песца клеточного разведения,ягоды и плоды растений вида vaccinium,яйца куриные в скорлупе свежие,ячмень
December 2023,NaN,36641.0,NaN,NaN,16478.0,21459.0,10406.0,19401.0,11603.0,4456.0,...,NaN,NaN,NaN,NaN,NaN,1828.0,NaN,NaN,9885.0,9527.0
January 2024,NaN,35924.5,NaN,145977.7,17280.4,19657.8,10891.8,23963.1,13044.7,4364.7,...,NaN,NaN,NaN,10191.1,NaN,2638.9,NaN,NaN,9747.3,9441.7
February 2024,NaN,32938.9,NaN,152960.3,18381.6,14010.6,10944.0,26826.1,13778.4,4384.4,...,NaN,NaN,NaN,NaN,NaN,2682.8,NaN,NaN,9575.4,9445.7
March 2024,NaN,33224.6,NaN,NaN,20116.3,14185.7,10888.2,26327.0,13054.1,4782.1,...,NaN,NaN,NaN,NaN,NaN,2572.5,NaN,NaN,9235.3,9330.0
April 2024,NaN,34392.7,NaN,143433.8,20987.6,14808.6,10712.6,25032.7,13630.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2092.6,NaN,NaN,9185.8,9436.7


In [136]:
cp_producers['пшеница']

January 2021      14810.0
February 2021     14974.0
March 2021        14095.0
April 2021        13317.0
May 2021          13140.0
June 2021         13415.0
July 2021         13179.0
August 2021       12826.0
September 2021    14173.0
October 2021      14705.0
November 2021     14936.0
December 2021     15133.0
January 2022      15360.0
February 2022     15501.0
March 2022        15267.0
April 2022        15982.0
May 2022          16164.0
June 2022         15495.0
July 2022         14504.0
August 2022       12575.0
September 2022    11209.0
October 2022      10584.0
November 2022     10823.0
December 2022     10754.0
January 2023      11266.0
February 2023     11530.0
March 2023        11351.0
April 2023        10928.0
May 2023          10350.0
June 2023         10096.0
July 2023         10178.0
August 2023       11411.0
September 2023    11063.0
October 2023      10885.0
November 2023     10399.0
December 2023     10295.0
January 2024      10376.7
February 2024     10470.7
March 2024  

### Growth since January 2021

In [137]:
cp_producers_growth = pd.DataFrame()
for column in cp_producers.columns:
    cp_producers_growth = pd.concat(
        [cp_producers_growth, normalized_by_first(cp_producers[column], return_type='df')],
        axis=1)

In [138]:
cp_producers_growth.head()

,nan,бобы соевые,виноград,волокно льна-долгунца,горох сушеный,гречиха,зерновые и зернобобовые культуры,капуста всех сортов,картофель,корнеплоды свеклы сахарной,...,томаты (помидоры) открытого грунта,тритикале,чеснок,"шерсть стриженая немытая овец и коз, включая стриженую шерсть, промытую руном",шкурки невыделанные лисицы клеточного разведения,шкурки невыделанные норки клеточного разведения,шкурки невыделанные песца клеточного разведения,ягоды и плоды растений вида vaccinium,яйца куриные в скорлупе свежие,ячмень
January 2021,NaN,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,NaN,1.000000,NaN,1.0,NaN,1.000000,NaN,NaN,1.000000,1.000000
February 2021,NaN,1.133424,NaN,1.169810,0.986931,1.069621,1.038000,1.076063,1.082079,0.86917,...,NaN,1.002509,NaN,NaN,NaN,1.272100,NaN,NaN,1.000696,1.139554
March 2021,NaN,1.301733,NaN,1.018458,1.023501,1.112811,0.990518,1.176690,1.190014,0.86389,...,NaN,1.169160,NaN,NaN,NaN,1.219436,NaN,NaN,1.061619,1.177682
April 2021,NaN,1.366029,NaN,1.144932,1.143175,1.157743,0.969657,1.084449,1.164022,NaN,...,NaN,1.086103,NaN,NaN,NaN,1.498433,NaN,NaN,1.175457,1.174886
May 2021,NaN,1.396792,NaN,1.028317,1.235715,1.207711,0.961649,1.067677,1.229549,NaN,...,NaN,NaN,NaN,NaN,NaN,1.003762,NaN,NaN,1.072585,1.243311


#### <font color='#820A22'> Export Dataset to Excel and CSV

In [119]:
datasets_list = [
    pci_producers, prices_producers_growth, pci_producers2021, pci_producers2022,
    pci_producers2023, pci_producers2024
]

In [120]:
sheet_names_list = [
    'СХ по годам', 'СХ рост с 2021', 'СХ 2021',
    'СХ 2022', 'СХ 2023', 'СХ 2024'
]

In [121]:
for dataset, sheet in zip(datasets_list, sheet_names_list):
    saveit_excel(
    data=dataset,
    filename='prices-2021-2024-producers',
    path=path_excel_csv,
    sheet=sheet
)

'СХ по годам' sheet created in file 'prices-2021-2024-producers.xlsx'
'СХ рост с 2021' sheet created in file 'prices-2021-2024-producers.xlsx'
'СХ 2021' sheet created in file 'prices-2021-2024-producers.xlsx'
'СХ 2022' sheet created in file 'prices-2021-2024-producers.xlsx'
'СХ 2023' sheet created in file 'prices-2021-2024-producers.xlsx'
'СХ 2024' sheet created in file 'prices-2021-2024-producers.xlsx'


### Save Dataset

In [122]:
economics_data['prices_producers_food'] = pci_producers

In [123]:
economics_data['prices_producers_food_growth'] = prices_producers_growth

In [124]:
economics_data['prices_2021_food_producers'] = pci_producers2021

In [125]:
economics_data['prices_2022_food_producers'] = pci_producers2022

In [126]:
economics_data['prices_2023_food_producers'] = pci_producers2023

In [127]:
economics_data['prices_2024_food_producers'] = pci_producers2024

In [128]:
saveit(
    file=economics_data,
    name='economics_data',
    path=path_files
)

File 'economics_data.pkl' saved


### Execution time

In [129]:
stopwatch_stop(start)

Execution time: 0:00:37
